<a href="https://colab.research.google.com/github/exphon/exphon2026/blob/main/MFA_LJSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Forced align LJSpeech dataset using Montreal Forced Aligner (MFA)


**Note**: The notebook takes 20 minutes to finish.

**DATA**: https://keithito.com/LJ-Speech-Dataset/

Expected results:

![english_mfa](https://github.com/exphon/exphon2026/blob/main/fig/english_mfa.png?raw=1)

## STEP 1: miniconda 설치를 위한 `install_mfa.sh` 작성

In [ ]:
Montreal Forced Aligner

In [ ]:
%%writefile install_mfa.sh
#!/bin/bash

## a script to install Montreal Forced Aligner (MFA)

root_dir=${1:-/tmp/mfa}

# Clean up previous installation
if [ -d "$root_dir" ]; then
    echo "Removing existing MFA installation at $root_dir"
    rm -rf $root_dir
fi
mkdir -p $root_dir
cd $root_dir

# download miniconda3
wget -q --show-progress https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p $root_dir/miniconda3 -f

# Initialize conda for the current shell to enable 'conda activate' etc.
eval "$($root_dir/miniconda3/bin/conda shell.bash hook)"

# Accept Conda Terms of Service (toc)
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r

# Create MFA environment with a specific Python version (e.g., 3.9) for compatibility
conda create -n aligner python=3.9 -y

# Activate the environment
conda activate aligner

# Install Montreal Forced Aligner into the activated environment
conda install -c conda-forge montreal-forced-aligner -y

echo -e "\n======== DONE =========="
echo -e "\nTo activate MFA, run: source $root_dir/miniconda3/bin/activate aligner"
echo -e "\nTo delete MFA, run: rm -rf $root_dir"
echo -e "\nSee: https://montreal-forced-aligner.readthedocs.io/en/latest/aligning.html to know how to use MFA"

## STEP 2: mfa 설치

In [ ]:
# download and install mfa
INSTALL_DIR="/tmp/mfa" # path to install directory

!bash ./install_mfa.sh {INSTALL_DIR}
# The following command needs to be executed in a way that the conda environment is properly activated
# Using `bash -c "source ... && mfa ..."` ensures it runs in a single shell context
!bash -c "source {INSTALL_DIR}/miniconda3/bin/activate aligner && export MPLBACKEND=Agg && mfa align --help"

## STEP 3: 영어 데이터 LJSpeech 다운로드

In [ ]:
# download and unpack ljs dataset
! echo "download and unpack ljs dataset"
! mkdir -p ./ljs
! cd ./ljs && wget -q --show-progress https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
#  wget -P ./ljs -q --show-progress https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
! cd ./ljs && tar xjf LJSpeech-1.1.tar.bz2

## STEP 4: SOX 설치 (Resampling to 16KHz)

이 코드는 sox라는 오디오 처리 도구를 사용하여 .wav 파일을 변환합니다. --norm=-3은 오디오 볼륨을 -3 dBFS로 정규화하여 일관된 볼륨 수준을 유지합니다. -r 16k는 샘플링 속도를 16kHz로 설정하고, -c 1은 오디오를 모노 채널로 변환합니다. 마지막으로, pwd/wav/{}는 처리된 파일을 ./wav 디렉토리에 저장하도록 지정합니다.

In [ ]:
# install sox tool
!sudo apt install -q -y sox
# convert to 16k audio clips
!mkdir ./wav
!echo "normalize audio clips to sample rate of 16k"
!find ./ljs -name "*.wav" -type f -execdir sox --norm=-3 {} -r 16k -c 1 `pwd`/wav/{} \;
!echo "Number of clips" $(ls ./wav/ | wc -l)

## STEP 5: txt 퍄일 준비하기

In [ ]:
# create transcript files from metadata.csv
lines = open('./ljs/LJSpeech-1.1/metadata.csv', 'r').readlines()
from tqdm.auto import tqdm
for line in tqdm(lines):
  fn, _, transcript = line.strip().split('|')
  ident = fn
  open(f'./wav/{ident}.txt', 'w').write(transcript)

# this is an example transcript for LJ001-0001.wav
!cat ./wav/LJ001-0001.txt

In [ ]:
from IPython.display import Audio
Audio('./wav/LJ001-0001.wav')

## STEP 6: 영어 Acoustic model 및 Lexicon model 다운로드하기

In [ ]:
# download a pretrained english acoustic model, and english lexicon
!wget -q --show-progress https://github.com/MontrealCorpusTools/mfa-models/raw/main/acoustic/english.zip
!wget -q --show-progress http://www.openslr.org/resources/11/librispeech-lexicon.txt

In [ ]:
! head librispeech-lexicon.txt

### Lexicon 모델 정비

In [ ]:
# see: https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner/pull/480
import re
lexicon = open("librispeech-lexicon.txt").readlines()
with open("modified_librispeech-lexicon.txt", "w") as f:
    for line in lexicon:
        word, *phonemes = re.split(r"\s+", line.strip())
        phonemes = " ".join(phonemes)
        f.write(f"{word}\t{phonemes}\n")

## STEP 7: MFA로 align하기

In [ ]:
# FINALLY, align phonemes and speech
! source {INSTALL_DIR}/miniconda3/bin/activate aligner && \
  export MPLBACKEND=Agg && \
  mfa align -t ./temp -j 4 ./wav modified_librispeech-lexicon.txt ./english.zip ./ljs_aligned

# output files are at ./ljs_aligned
!echo "See output files at ./ljs_aligned"

제공된 MFA(Montreal Forced Aligner) 출력 로그를 바탕으로 강제 정렬(Forced Alignment) 진행 과정


- Corpus 정보 설정 및 로드 (Setting up corpus information..., Loading corpus from source files...): MFA가 정렬을 시작하기 위해 입력된 오디오 파일과 텍스트 스크립트 데이터를 준비하고 불러오는 초기 단계입니다.

- 코퍼스 통계 보고 (Found 1 speaker across 13100 files, average number of utterances per speaker: 13100.0): 입력된 데이터셋에서 1명의 화자(speaker)가 13,100개의 발화(utterance)를 가지고 있으며, 화자당 평균 발화 수가 13,100개임을 알려줌.

- 멀티프로세싱 작업 초기화 및 경고 (Initializing multiprocessing jobs... WARNING Number of jobs was specified as 4, but due to only having 1 speakers, MFA will only use 1 jobs. Use the --single_speaker flag if you would like to split utterances across jobs regardless of their speaker.): MFA는 여러 CPU 코어를 활용하여 작업을 병렬로 처리 가능. 원래 4개의 작업을 지정했지만, 화자가 1명뿐이므로 MFA는 1개의 작업만 사용. 만약 여러 화자가 없는 경우에도 작업을 분할하여 병렬 처리 효율을 높이고 싶다면 --single_speaker 플래그를 사용하라는 안내.

- 텍스트 정규화 (Normalizing text...): 스크립트 텍스트를 MFA가 처리하기 적합한 형태로 변환하는 과정. 구두점 제거, 대소문자 변환 등 언어 모델 학습에 필요한 전처리 과정을 수행.

- MFCC 생성 (Generating MFCCs...): 오디오 파일에서 음향 특징(Acoustic Features)을 추출하는 단계. MFCC(Mel-Frequency Cepstral Coefficients)는 음성 인식 및 처리에서 널리 사용되는 특징으로, 오디오의 스펙트럼 정보를 압축하여 표현.

- CMVN 계산 (Calculating CMVN...): MFCC 특징에 대해 CMVN(Cepstral Mean and Variance Normalization)을 적용하는 단계. 이는 화자의 음성 특징이나 녹음 환경의 차이로 인한 편향을 줄여 음향 모델의 성능을 향상시키는 데 도움.

- 최종 특징 생성 (Generating final features...): 정규화된 MFCC를 포함하여 음향 모델 학습 및 정렬에 사용될 최종 특징 벡터를 준비.

- 코퍼스 분할 (Creating corpus split...): MFA 내부적으로 데이터를 처리하기 위해 코퍼스를 분할하는 과정.

- 학습 그래프 컴파일 (Compiling training graphs...): 음향 모델과 lexicon을 기반으로 정렬에 필요한 FST를 생성하는 단계. 음성과 텍스트 간의 가능한 매핑 경로 정의

- 1차 정렬 수행 (Performing first-pass alignment... Generating alignments...): 첫 번째 정렬 시도. 초기 음향 모델을 사용하여 오디오와 텍스트 간의 대략적인 시간 정렬을 수행

- 화자 적응을 위한 fMLLR 계산 (Calculating fMLLR for speaker adaptation...): 1차 정렬 결과를 바탕으로 fMLLR(feature-space Maximum Likelihood Linear Regression) 변환을 계산하여 음향 모델을 현재 화자에게 더 잘 맞도록 적응.

- 2차 정렬 수행 (Performing second-pass alignment... Generating alignments...): fMLLR 기반 화자 적응이 적용된 모델을 사용하여 더 정확한 정렬을 다시 수행

- 정렬 격자에서 음소 및 단어 정렬 수집 (Collecting phone and word alignments from alignment lattices...): 2차 정렬 결과물인 alignment lattices에서 각 단어와 음소의 시작 및 끝 시간을 추출

- 정렬 품질 분석 (Analyzing alignment quality...): 정렬된 결과의 품질을 평가

- TextGrid로 정렬 내보내기 (Exporting alignment TextGrids to ljs_aligned... Finished exporting TextGrids to ljs_aligned!): 최종 정렬 결과를 Praat 소프트웨어와 같은 도구에서 열 수 있는 TextGrid 형식으로 지정된 출력 디렉토리(ljs_aligned)에 저장

- 완료 (Done! Everything took 1053.207 seconds)

요약하자면, MFA는 오디오 데이터에서 음향 특징을 추출하고, 텍스트를 전처리한 다음, 여러 단계의 음향 모델 기반 정렬(화자 적응 포함)을 통해 오디오 내에서 각 단어와 음소의 정확한 시작 및 끝 시간을 찾아 TextGrid 파일로 저장하는 과정을 거칩니다.



## STEP 8: zip 파일로 압축하여 결과물 다운로드하기

In [ ]:
!zip -r ljs_aligned.zip ./ljs_aligned

In [ ]:
from google.colab import files
files.download('ljs_aligned.zip')

# References

- https://gist.github.com/NTT123/12264d15afad861cb897f7a20a01762e (본 tutorial에서 사용한 원래의 colab 코드)
- [Montreal Forced Aligner documentation 홈페이지](https://montreal-forced-aligner.readthedocs.io/en/latest/)
- [MFA github](https://github.com/MontrealCorpusTools/Montreal-Forced-Aligner)
